# Automatic Differentiation 

(Credits: Alan Edelman, 2017)


In our previous notebooks, we've had to take derivatives of the objective function wrt different parameters. Calculating the derivative by pen and paper is obviously quite taxing, so there must be a better way, instead of using tables. 

  <img src="http://www2.bc.cc.ca.us/resperic/math6a/lectures/ch5/1/IntegralTable.gif" width="190">
  
  Or even using finite differences, like one learns in a numerical computing class, which often has high error and is relatively slow.
  
<img src="http://image.mathcaptain.com/cms/images/122/Diff%202.png" width="150">



## Babylonian sqrt

Consider a simple example, computation of `sqrt(x)`, where for me how autodiff works came as both a mathematical surprise, and a computing wonder.  The example is  the Babylonian algorithm, known to man for millenia, to compute sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.
 
 Each iteration has one add and two divides. For illustration purposes, 10 iterations suffice.

In [1]:
function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  

Babylonian (generic function with 1 method)

In [2]:
Babylonian(π), √π   # \pi + <tab> , and \sqrt + <tab>  

(1.7724538509055159, 1.7724538509055159)

Side note: Julia supports unicode symbols

In [3]:
α = 7 
π

π = 3.1415926535897...

Check that it works:

In [4]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

(1.414213562373095, 1.4142135623730951)

In [5]:
using Plots
plotly()

Plots.PlotlyBackend()

In [6]:
i = 0:.01:49

plot([x->Babylonian(x,N=i) for i=1:5],i,label=["Iteration $j" for i=1:1,j=1:5])

plot!(sqrt,i,c="black",label="sqrt",
      title = "Those Babylonians really knew how to √")

## ...and now the derivative, almost by magic

In [7]:
struct D <: Number  # D is a function-derivative pair
    f::Tuple{Float64,Float64}
end

Sum Rule: (x+y)' = x' + y' <br>
Quotient Rule: (x/y)' = (yx'-xy') / y^2

In [8]:
import Base: +, /, convert, promote_rule
+(x::D, y::D) = D(x.f .+ y.f)
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D}, x::Real) = D((x,zero(x)))
promote_rule(::Type{D}, ::Type{<:Number}) = D

promote_rule (generic function with 140 methods)

In [9]:
x=π; Babylonian(D((x,1))), (√x,.5/√x)

(D((1.7724538509055159, 0.28209479177387814)), (1.7724538509055159, 0.28209479177387814))

In [10]:
i = .2:.01:49
plot([x->Babylonian(D((x,1.0)),N=i).f[2] for i=1:5],i)
plot!(x->.5/√x,i,c="black",label="d(sqrt(x))/dx",
    title = " Babylonians Differentiated")

## It just works!

How does it work?  We will explain in a moment.  Right now marvel that it does.  Note we did not
import any autodiff package.  Everything is just basic vanilla Julia.

## The assembler

Most folks don't read assembler, but one can see that it is short.

In [11]:
@inline function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  
@code_native(Babylonian(D((2,1))))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[11]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	movsd	(%rsi), %xmm0           ## xmm0 = mem[0],zero
	movsd	8(%rsi), %xmm8          ## xmm8 = mem[0],zero
	movabsq	$4838989528, %rax       ## imm = 0x1206D1ED8
	movsd	(%rax), %xmm4           ## xmm4 = mem[0],zero
	addsd	%xmm0, %xmm4
	xorpd	%xmm2, %xmm2
	movapd	%xmm8, %xmm1
	addsd	%xmm2, %xmm1
	addsd	%xmm1, %xmm1
	mulsd	%xmm4, %xmm2
	subsd	%xmm2, %xmm1
	movabsq	$4838989536, %rcx       ## imm = 0x1206D1EE0
	mulsd	(%rcx), %xmm4
	movabsq	$4838989544, %rdx       ## imm = 0x1206D1EE8
	mulsd	(%rdx), %xmm1
	movl	$9, %eax
	movabsq	$4838989552, %rsi       ## imm = 0x1206D1EF0
	movsd	(%rsi), %xmm9           ## xmm9 = mem[0],zero
	movsd	(%rcx), %xmm5           ## xmm5 = mem[0],zero
	movsd	(%rdx), %xmm6           ## xmm6 = mem[0],zero
	nopw	%cs:(%rax,%rax)
Source line: 17
L128:
	movapd	%xmm0, %xmm7
	divsd	%xmm4, %xmm7
	movapd	%xmm4, %xmm2
	mulsd	%xmm8, %xmm2
	movapd	%xmm1, %xmm3
	mulsd	%xm

## Symbolically

We haven't yet explained how it works, but it may be of some value to understand that the below is mathematically
equivalent, though not what the computation is doing.

Notice in the below that Babylonian works on SymPy symbols.

In [36]:
using SymPy

INFO: Precompiling module SymPy.


LoadError: InitError: [91mFailed to import required Python module sympy.

For automated sympy installation, try configuring PyCall to use the Conda.jl package's Python "Miniconda" distribution within Julia. Relaunch Julia and run:
    ENV["PYTHON"]=""
    Pkg.build("PyCall")
before trying again.

The pyimport exception was: PyError (ccall(@pysym(:PyImport_ImportModule), PyPtr, (Cstring,), name)

The Python package sympy could not be found by pyimport. Usually this means
that you did not install sympy in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/usr/local/opt/python/bin/python2.7

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the sympy module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-la

In [37]:
x = symbols("x")
display("Iterations as a function of x")
for k = 1:5
 display( simplify(Babylonian(x,N=k)))
end

display("Derivatives as a function of x")
for k = 1:5
 display(simplify(diff(simplify(Babylonian(x,N=k)),x)))
end

LoadError: [91mUndefVarError: symbols not defined[39m

## How autodiff is getting the answer
Let us by hand take the "derivative" of the Babylonian iteration with respect to x. Specifically t′=dt/dx

In [15]:
function dBabylonian(x; N = 10) 
    t = (1+x)/2
    t′ = 1/2
    for i = 1:N;  
        t = (t+x/t)/2; 
        t′= (t′+(t-x*t′)/t^2)/2; 
    end    
    t′

end  

dBabylonian (generic function with 1 method)

In [16]:
x = π; dBabylonian(x), .5/√x

(0.2820947917738782, 0.28209479177387814)

What just happened?  Answer: We created an iteration by hand for t′ given our iteration for t. Then we ran the iteration alongside the iteration for t.

In [17]:
Babylonian(D((x,1)))

D((1.7724538509055159, 0.28209479177387814))

How did this work?  It created the same derivative iteration, using very general rules that are set once and need not be written by hand.

**Important**: The derivative is substituted before the JIT compiler, and thus efficient compiled code is executed.

## Dual Number Notation

Instead of D(a,b) we can write a + b ϵ, where ϵ satisfies ϵ^2=0.  (Some people like to recall imaginary numbers where an i is introduced with i^2=-1.) 

Others like to think of how engineers just drop the O(ϵ^2) terms.

The four rules are

$ (a+b\epsilon) \pm (c+d\epsilon) = (a+c) \pm (b+d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/d^2 \epsilon $


In [18]:
Base.show(io::IO,x::D) = print(io,x.f[1]," + ",x.f[2]," ϵ")

In [19]:
# Add the last two rules
import Base: -,*
-(x::D, y::D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))

* (generic function with 213 methods)

In [20]:
D((2,1)) ^2

4.0 + 4.0 ϵ

In [21]:
ϵ = D((0,1))

0.0 + 1.0 ϵ

In [22]:
@code_native(ϵ^2)

	.section	__TEXT,__text,regular,pure_instructions
Filename: intfuncs.jl
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rbx
	subq	$24, %rsp
	movq	%rdi, %rbx
Source line: 195
	movabsq	$power_by_squaring, %rax
	leaq	-24(%rbp), %rdi
	callq	*%rax
	movq	-24(%rbp), %rax
	movq	-16(%rbp), %rcx
	movq	%rcx, 8(%rbx)
	movq	%rax, (%rbx)
	movq	%rbx, %rax
	addq	$24, %rsp
	popq	%rbx
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [23]:
ϵ * ϵ 

0.0 + 0.0 ϵ

In [24]:
ϵ^2

0.0 + 0.0 ϵ

In [25]:
1/(1+ϵ)  # Exact power series:  1-ϵ+ϵ²-ϵ³-...

1.0 + -1.0 ϵ

In [26]:
(1+ϵ)^10 ## Note this just works (we didn't train powers)!!

1.0 + 10.0 ϵ

## Generalization to arbitrary roots

In [27]:
function nthroot(x, n=2; t=1, N = 10) 
    for i = 1:N;   t += (x/t^(n-1)-t)/n; end   
    t
end  

nthroot (generic function with 2 methods)

In [28]:
nthroot(2,3), ∛2 # take a cube root

(1.2599210498948732, 1.2599210498948732)

In [29]:
nthroot(7,12), 7^(1/12)

(1.1760474285795146, 1.1760474285795146)

In [30]:
x = 17.0
nthroot( x+ϵ,3), ∛x, 1/x^(2/3)/3

(2.571281590658235 + 0.05041728609133794 ϵ, 2.571281590658235, 0.05041728609133795)

## Forward Diff
Now that you understand it, you can use the official package

In [31]:
using ForwardDiff

In [32]:
ForwardDiff.derivative(sqrt, 2)

0.35355339059327373

In [33]:
ForwardDiff.derivative(Babylonian, 2)

0.35355339059327373

In [34]:
@which ForwardDiff.derivative(sqrt, 2)

derivative(f::F, x::R) where {F, R<:Real} in ForwardDiff at /Users/ranjan/.julia/v0.6/ForwardDiff/src/derivative.jl:13